In [3]:
import findspark
# provide path to your spark directory directly
findspark.init("/home/canwill/spark2")
import pyspark
from __future__ import print_function

import sys

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession



In [4]:
def getSparkSessionInstance(sparkConf):
    if ('sparkSessionSingletonInstance' not in globals()):
        globals()['sparkSessionSingletonInstance'] = SparkSession\
            .builder\
            .config(conf=sparkConf)\
            .getOrCreate()
    return globals()['sparkSessionSingletonInstance']



In [5]:
if __name__ == "__main__":
    host = 'localhost'
    port = '9999'
    sc = SparkContext(appName="PythonSqlNetworkWordCount")
    ssc = StreamingContext(sc, 1)

    # Create a socket stream on target ip:port and count the
    # words in input stream of \n delimited text (eg. generated by 'nc')
    lines = ssc.socketTextStream(host, int(port))
    words = lines.flatMap(lambda line: line.split(" "))


In [6]:
def process(time, rdd):
    try:
        # Get the singleton instance of SparkSession
        spark = getSparkSessionInstance(rdd.context.getConf())

        # Convert RDD[String] to RDD[Row] to DataFrame
        rowRdd = rdd.map(lambda w: Row(word=w))
        wordsDataFrame = spark.createDataFrame(rowRdd)

        # Creates a temporary view using the DataFrame.
        wordsDataFrame.createOrReplaceTempView("words")

        # Do word count on table using SQL and print it
        wordCountsDataFrame = \
            spark.sql("select word, count(*) as total from words group by word")
        wordCountsDataFrame.show()
    except:
        pass

In [ ]:
words.foreachRDD(process)
ssc.start()
ssc.awaitTermination()

+----+-----+
|word|total|
+----+-----+
|  Hi|    1|
| Hey|    1|
+----+-----+

+----+-----+
|word|total|
+----+-----+
| How|    1|
|you?|    1|
| are|    1|
+----+-----+

+-----+-----+
| word|total|
+-----+-----+
|   is|    1|
|  How|    1|
|life?|    1|
+-----+-----+

+-------------+-----+
|         word|total|
+-------------+-----+
|          One|    1|
|          set|    1|
|       Sidney|    1|
|       today.|    1|
|           by|    4|
|       shape,|    1|
|   mythology.|    1|
|          'W'|    1|
|          was|    1|
|          one|    1|
|    celestial|    1|
|           in|    3|
| illustration|    1|
|     Urania's|    1|
|  illustrated|    1|
|        after|    1|
|           is|    2|
|   Cassiopeia|    2|
|constellation|    1|
|           it|    1|
+-------------+-----+
only showing top 20 rows

+--------------------+-----+
|                word|total|
+--------------------+-----+
|                 not|    1|
|              world.|    1|
|                  by|    1|
| 